# CSE 432 Midterm Report - Owen Fankhauser

## Imports & Setup

Import appropriate Libraries.

In [49]:
import numpy as np
import pandas as pd
import glob
import os
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

Save global values.

In [50]:
# Sensor Frequencies
AcceleratorHz = 100
GyroscopeHz = 100
MagnetometerHZ = 20
PressureSensorHz = 7

# We know how many files should be in this data package
expectedFileCount = 520

## Data Preparation

### 1. Load data

Data paths are gathered using glob, parsed into individial reads, and appended into an array. The array is then parsed into a DataFrame "df". The file path is collected as data as well for later use. This data is declared globally. *7-8 seconds to complete.*

In [51]:
filePaths = glob.glob("CourseProjectDataSet1/**/*.csv", recursive=True)

dataList = []
for file in filePaths:
    df = pd.read_csv(file)
    df["file_path"] = file # stored for later
    dataList.append(df)

data = pd.concat(dataList, ignore_index=True)

### 2. Extract Metadata

This function uses to Regex to parse individual file names into metadata that is appended to the DataFrame. *3-4 minutes to complete.*

In [52]:
def extractMetadata(filePaths):
    # Divvy up the file name and search using Regex for groups.
    fileName = os.path.basename(filePaths)
    fileName
    pattern = r"([\w]+)_([\w]+)_(\d{4}-\d{2}-\d{2})T([\d.]+)_(\w+)_([\w]+)_(\d+\.\d+)Hz"
    match = re.search(pattern, fileName)

    # Extract user from the path since it won't match the Regex search.
    user = filePaths.split(os.sep)[2]  
    
    # Only returns if there is a match. Otherwise stop program.
    if match:
        expID = match.group(1)  # Experiment ID
        device = match.group(2)  # Irrelevant data
        date = match.group(3)  # Date
        time = match.group(4) # Time
        sensor_type = match.group(6)  # Sensor type (Accelerometer, etc)
        frequency = float(match.group(7))  # Frequency in Hz
        
        return pd.Series([expID, date, time, frequency, sensor_type, user])
    else :
        raise Exception("Incorrect file name: " + fileName)
    
# Iterate through all file paths and extract data. 
data[["expID", "date", "time", "frequency", "sensor_type", "user"]] = data["file_path"].apply(extractMetadata)

### 3. Data Verification

First, make sure all files are accounted for. We know there are 520 files in this particular example. We also can check if there are more than 520 files present to eliminate duplicate files. Our expected file count can be changed to fit other data, or set to -1 to disable this check.
 *Later iterations may include a menu on start up to enter how many files we expect to add to the dataset*

In [53]:
if expectedFileCount > 0 :
    if len(filePaths) < expectedFileCount or len(filePaths) > expectedFileCount:
        raise Exception((filePaths) + " files found. Expected: " + expectedFileCount + " files.")
data

,epoch (ms),time (-13:00),elapsed (s),x-axis (g),y-axis (g),z-axis (g),file_path,x-axis (deg/s),y-axis (deg/s),z-axis (deg/s),x-axis (T),y-axis (T),z-axis (T),pressure (Pa),expID,date,time,frequency,sensor_type,user
0,1568482903468,2019-09-14T13:41:43.468,0.000,0.715,0.742,-0.200,CourseProjectDataSet1\DataSet1\User1\1_MetaWea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2019-09-14,13.41.43.335,100.00,Accelerometer,User1
1,1568482903478,2019-09-14T13:41:43.478,0.010,0.722,0.755,-0.201,CourseProjectDataSet1\DataSet1\User1\1_MetaWea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2019-09-14,13.41.43.335,100.00,Accelerometer,User1
2,1568482903488,2019-09-14T13:41:43.488,0.020,0.718,0.766,-0.197,CourseProjectDataSet1\DataSet1\User1\1_MetaWea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2019-09-14,13.41.43.335,100.00,Accelerometer,User1
3,1568482903498,2019-09-14T13:41:43.498,0.030,0.709,0.788,-0.196,CourseProjectDataSet1\DataSet1\User1\1_MetaWea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2019-09-14,13.41.43.335,100.00,Accelerometer,User1
4,1568482903508,2019-09-14T13:41:43.508,0.040,0.696,0.798,-0.196,CourseProjectDataSet1\DataSet1\User1\1_MetaWea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2019-09-14,13.41.43.335,100.00,Accelerometer,User1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3414945,1574109094723,2019-11-18T15:31:34.723,36.365,NaN,NaN,NaN,CourseProjectDataSet1\DataSet1\User9\99_MetaWe...,NaN,NaN,NaN,NaN,NaN,NaN,97507.094,99,2019-11-18,15.30.58.254,7.33,Pressure,User9
3414946,1574109094859,2019-11-18T15:31:34.859,36.501,NaN,NaN,NaN,CourseProjectDataSet1\DataSet1\User9\99_MetaWe...,NaN,NaN,NaN,NaN,NaN,NaN,97513.609,99,2019-11-18,15.30.58.254,7.33,Pressure,User9
3414947,1574109094994,2019-11-18T15:31:34.994,36.636,NaN,NaN,NaN,CourseProjectDataSet1\DataSet1\User9\99_MetaWe...,NaN,NaN,NaN,NaN,NaN,NaN,97513.125,99,2019-11-18,15.30.58.254,7.33,Pressure,User9
3414948,1574109095130,2019-11-18T15:31:35.130,36.772,NaN,NaN,NaN,CourseProjectDataSet1\DataSet1\User9\99_MetaWe...,NaN,NaN,NaN,NaN,NaN,NaN,97510.516,99,2019-11-18,15.30.58.254,7.33,Pressure,User9


### 4. Map Activity Lables

Create an activities DataFrame to merge with the existing data DataFrame. It will be used to map expid to activities to activity number and type. *ChatGPT and Gemini were both used on this step to shorten the lengthy process of writing all of this data out as well as to double check for accuracy against the Data Collection Sheet.*

First, build the Data Colelction Sheet Database (AI assisted) 

In [58]:
activity_data = { # New array to record Data Collection Sheet data
    'expID': [
        1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, "Missing", 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, "Missing", 127, 128, "Missing", 130, 131, 132
    ],
    'Activity No.': [
        1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11
    ],
    'Activity Type': [
        "Sitting", "Sitting", "Using computer", "Using computer", "While sitting", "While sitting", "Sitting", "Standing", "Missing", "Missing", "Missing",
        "Sitting", "Sitting", "Using computer", "Using computer", "While sitting", "While sitting", "Sitting", "Standing", "Missing", "Missing", "Missing",
        "Sitting", "Sitting", "Using computer", "Using computer", "While sitting", "While sitting", "Sitting", "Standing", "Missing", "Missing", "Missing",
        "Sitting", "Sitting", "Using computer", "Using computer", "While sitting", "While sitting", "Sitting", "Standing", "Missing", "Missing", "Missing",
        "Sitting", "Sitting", "Using computer", "Using computer", "While sitting", "While sitting", "Sitting", "Standing", "Missing", "Missing", "Missing",
        "Sitting", "Sitting", "Using computer", "Using computer", "While sitting", "While sitting", "Sitting", "Standing", "Missing", "Missing", "Missing",
        "Sitting", "Sitting", "Using computer", "Using computer", "While sitting", "While sitting", "Sitting", "Standing", "Missing", "Missing", "Missing",
        "Sitting", "Sitting", "Using computer", "Using computer", "While sitting", "While sitting", "Sitting", "Standing", "Missing", "Missing", "Missing",
        "Sitting", "Sitting", "Using computer", "Using computer", "While sitting", "While sitting", "Sitting", "Standing", "Missing", "Missing", "Missing",
        "Sitting", "Sitting", "Using computer", "Using computer", "While sitting", "While sitting", "Sitting", "Standing", "Missing", "Missing", "Missing",
        "Sitting", "Sitting", "Using computer", "Using computer", "While sitting", "While sitting", "Sitting", "Standing", "Missing", "Missing", "Missing",
        "Sitting", "Sitting", "Using computer", "Using computer", "While sitting", "While sitting", "Sitting", "Standing", "Missing", "Missing", "Missing"
    ],
    'Activity Details': [
        "Reading a book", "Writing on a notebook", "Typing", "Browsing", "Moving head, body", "Moving chair", "Stand up from sitting", "Standing", "Walking", "Running", "Taking stairs",
        "Reading a book", "Writing on a notebook", "Typing", "Browsing", "Moving head, body", "Moving chair", "Stand up from sitting", "Standing", "Walking", "Running", "Taking stairs",
        "Reading a book", "Writing on a notebook", "Typing", "Browsing", "Moving head, body", "Moving chair", "Stand up from sitting", "Standing", "Walking", "Running", "Taking stairs",
        "Reading a book", "Writing on a notebook", "Typing", "Browsing", "Moving head, body", "Moving chair", "Stand up from sitting", "Standing", "Walking", "Running", "Taking stairs",
        "Reading a book", "Writing on a notebook", "Typing", "Browsing", "Moving head, body", "Moving chair", "Stand up from sitting", "Standing", "Walking", "Running", "Taking stairs",
        "Reading a book", "Writing on a notebook", "Typing", "Browsing", "Moving head, body", "Moving chair", "Stand up from sitting", "Standing", "Walking", "Running", "Taking stairs",
        "Reading a book", "Writing on a notebook", "Typing", "Browsing", "Moving head, body", "Moving chair", "Stand up from sitting", "Standing", "Walking", "Running", "Taking stairs",
        "Reading a book", "Writing on a notebook", "Typing", "Browsing", "Moving head, body", "Moving chair", "Stand up from sitting", "Standing", "Walking", "Running", "Taking stairs",
        "Reading a book", "Writing on a notebook", "Typing", "Browsing", "Moving head, body", "Moving chair", "Stand up from sitting", "Standing", "Walking", "Running", "Taking stairs",
        "Reading a book", "Writing on a notebook", "Typing", "Browsing", "Moving head, body", "Moving chair", "Stand up from sitting", "Standing", "Walking", "Running", "Taking stairs",
        "Reading a book", "Writing on a notebook", "Typing", "Browsing", "Moving head, body", "Moving chair", "Stand up from sitting", "Standing", "Walking", "Running", "Taking stairs",
        "Reading a book", "Writing on a notebook", "Typing", "Browsing", "Moving head, body", "Moving chair", "Stand up from sitting", "Standing", "Walking", "Running", "Taking stairs"
    ]
}

Second, clean up the Data, make it a DataFrame, and merge it with the existing DataFrame. *7-8 seconds to complete.*

In [ ]:
# Make sure all rows are equal
max_length = max(len(activity_data[key]) for key in activity_data)

# Clean up missing Activity Type data
for key in activity_data:
    while len(activity_data[key]) < max_length:
        activity_data[key].append(None)  # Add None for missing values

# Convert to DataFrame and clean up missing data
activity_df = pd.DataFrame(activity_data)
activity_df.replace("Missing", None, inplace=True) # Replace "missing" with None

# Confirms expIDs are all strings in both DataFrames
data['expID'] = data['expID'].astype(str)
activity_df['expID'] = activity_df['expID'].astype(str)


# Finally, merge based on expIDs
data = data.merge(activity_df, on='expID', how='left')

# Clean up duplicate columns created in merge

# TODO Figure out why these duplicate generate. Merge Left Failure? #

data = data.rename(columns={'Activity No._y': 'Activity No.',
                            'Activity Type_y': 'Activity Type',
                            'Activity Details_y': 'Activity Details',
                            'Activity No._x': 'Activity No.',
                            'Activity Type_x': 'Activity Type',
                            'Activity Details_x': 'Activity Details',
                            'Activity No._activity': 'Activity No.',
                            'Activity Type_activity': 'Activity Type',
                            'Activity Details_activity': 'Activity Details'
                            })
data = data.loc[:, ~data.columns.duplicated()]
data

,epoch (ms),time (-13:00),elapsed (s),x-axis (g),y-axis (g),z-axis (g),file_path,x-axis (deg/s),y-axis (deg/s),z-axis (deg/s),...,pressure (Pa),expID,date,time,frequency,sensor_type,user,Activity No.,Activity Type,Activity Details
0,1568482903468,2019-09-14T13:41:43.468,0.000,0.715,0.742,-0.200,CourseProjectDataSet1\DataSet1\User1\1_MetaWea...,NaN,NaN,NaN,...,NaN,1,2019-09-14,13.41.43.335,100.00,Accelerometer,User1,1.0,Sitting,Reading a book
1,1568482903478,2019-09-14T13:41:43.478,0.010,0.722,0.755,-0.201,CourseProjectDataSet1\DataSet1\User1\1_MetaWea...,NaN,NaN,NaN,...,NaN,1,2019-09-14,13.41.43.335,100.00,Accelerometer,User1,1.0,Sitting,Reading a book
2,1568482903488,2019-09-14T13:41:43.488,0.020,0.718,0.766,-0.197,CourseProjectDataSet1\DataSet1\User1\1_MetaWea...,NaN,NaN,NaN,...,NaN,1,2019-09-14,13.41.43.335,100.00,Accelerometer,User1,1.0,Sitting,Reading a book
3,1568482903498,2019-09-14T13:41:43.498,0.030,0.709,0.788,-0.196,CourseProjectDataSet1\DataSet1\User1\1_MetaWea...,NaN,NaN,NaN,...,NaN,1,2019-09-14,13.41.43.335,100.00,Accelerometer,User1,1.0,Sitting,Reading a book
4,1568482903508,2019-09-14T13:41:43.508,0.040,0.696,0.798,-0.196,CourseProjectDataSet1\DataSet1\User1\1_MetaWea...,NaN,NaN,NaN,...,NaN,1,2019-09-14,13.41.43.335,100.00,Accelerometer,User1,1.0,Sitting,Reading a book
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3414945,1574109094723,2019-11-18T15:31:34.723,36.365,NaN,NaN,NaN,CourseProjectDataSet1\DataSet1\User9\99_MetaWe...,NaN,NaN,NaN,...,97507.094,99,2019-11-18,15.30.58.254,7.33,Pressure,User9,11.0,None,Taking stairs
3414946,1574109094859,2019-11-18T15:31:34.859,36.501,NaN,NaN,NaN,CourseProjectDataSet1\DataSet1\User9\99_MetaWe...,NaN,NaN,NaN,...,97513.609,99,2019-11-18,15.30.58.254,7.33,Pressure,User9,11.0,None,Taking stairs
3414947,1574109094994,2019-11-18T15:31:34.994,36.636,NaN,NaN,NaN,CourseProjectDataSet1\DataSet1\User9\99_MetaWe...,NaN,NaN,NaN,...,97513.125,99,2019-11-18,15.30.58.254,7.33,Pressure,User9,11.0,None,Taking stairs
3414948,1574109095130,2019-11-18T15:31:35.130,36.772,NaN,NaN,NaN,CourseProjectDataSet1\DataSet1\User9\99_MetaWe...,NaN,NaN,NaN,...,97510.516,99,2019-11-18,15.30.58.254,7.33,Pressure,User9,11.0,None,Taking stairs
